# Runner类
## Motivation
`Runner`三种运行方式：
1. `Runner.run_sync()`同步运行：
- 直接运行，不通过异步处理，适合直接获取结果（不关注过程）
- 启动任务，单线程堵塞程序的执行，知道Agent完成并返回最终结果

2. `Runner.run()`异步运行：
- 性能效率高，适合处理高并发或者与其他异步I/O继承的场景，也就是Jupyter Notebook的适配运行模式
- 启动任务后，立即返回一个可等待的（Awaitable）对象；需要使用`await`等待其完成，在其等待期间，时间循环处理其他任务。

3. `Runner.run_streamed()`流式异步运行：
- 最佳用户体验运行形式，交互式应用中，呈现出来就是事实Token生成，而不是等待响应结果。
- 提供实时监控Agent内部的详细步骤。
- 启动任务后，立即返回一个特色的`runResultStreaming`对象。你不能直接`await`这个对象（如果使用`await Runner.run_streamed()`则会报错），使用`async for`来异步迭代它的事件流

## Definition
- `Runner.run_streamed(agent, input, ...)`：这是一个异步方法，当你调用它时，它会启动Agent任务，并立刻返回一个`RunResultStreaming`对象，不会等待任务完成。

- `RunResultStreaming`对象：可以把它想象成一个“事件发射器”或“直播信号源”。它本身不是最终结果，而是一个**异步迭代器 (Async Iterator)**。你需要用`async for event in result.stream_events()`:来不断地从中“接收”事件，直到任务完成。

- 事件 (Events)：`stream_events()`产生的每个`event`都是一个描述Agent运行过程中某个时刻发生的事情的对象。它有一个type属性（如"`raw_response_event`", "`agent_updated_stream_event`"等）和一个`data`或`run_item_stream_event`属性，包含了**具体的事件信息**。

## Design&Implementation
处理流式输出

示例1:基本流式文本输出
捕获并打印大模型生成的文本流
```python
# ... (LLM配置, Agent定义等保持不变) ...

async def main():
    # 1. 调用 run_streamed，获取流对象 result
    result = Runner.run_streamed(agent, "给我讲个程序员相亲的笑话")
    
    # 2. 使用 async for 异步迭代事件流
    async for event in result.stream_events():
        # 3. 判断事件类型是否为原始响应事件 (模型正在输出文本)
        #    并且事件数据是 ResponseTextDeltaEvent (表示文本片段)
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            # 4. 打印接收到的文本片段 (event.data.delta)
            #    end="" 阻止打印换行符，flush=True 确保立即显示
            print(event.data.delta, end="", flush=True)

# ... (if __name__ == "__main__": asyncio.run(main())) ...
```
- `raw_response_event`代表LLM原始输出的“字符流”
- `event_data.delta`就是每次收到那一小段文本，通过`print(...,end="",flush=True)`，打印实现打字机效果

示例2:捕获更多内部事件
捕获Agent内部运作
```python
# ... (LLM配置, Agent定义, how_many_jokes工具定义等) ...

async def main():
    # ... (Agent 定义) ...
    agent = Agent(
        name="Joker",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        model=llm,
    )

    result = Runner.run_streamed(agent, input="Hello")
    print("=== Run starting ===")

    async for event in result.stream_events():
        # 忽略原始文本流事件
        if event.type == "raw_response_event":
            continue
        # Agent状态更新事件 
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # 运行项事件 (最丰富的信息来源)
        elif event.type == "run_item_stream_event":
            item = event.item # 获取事件的具体项目
            if item.type == "tool_call_item":
                print("-- Tool was called") # 知道工具被调用了
            elif item.type == "tool_call_output_item":
                print(f"-- Tool output: {item.output}") # 看到工具返回了什么
            elif item.type == "message_output_item":
                # 使用 ItemHelpers 提取格式化的消息文本
                print(f"-- Message output:\n {ItemHelpers.text_message_output(item)}")
            else:
                pass # 忽略其他类型的 run_item_stream_event

    print("=== Run complete ===")

# ... (if __name__ == "__main__": asyncio.run(main())) ...
```
`event.type`中：
- `run_item_stream_event`：包含了Agent运行过程中的所有重要事件，如工具调用、消息输出等。
- `item.type`：表示事件的类型
    - "`tool_call_item`"，工具调用事件
    - "`tool_call_output_item`"，工具调用输出事件
    - "`message_output_item`"，消息输出事件
- `item.output`：如果是工具调用输出事件，这里会包含工具返回的结果。
- `ItemHelpers.text_message_output(item)`：辅助函数，用于格式化消息，从`message_output_item`中提取用户友好的文本内容。

>示例缺乏展示`tool_call_output_item`，并没有有效调用工具且推出结果，应该优化。
>建议调整`input="Tell me some jokes!"`，包含工具调用。
>修改所调用工具如下：
```python
def how_many_jokes() -> int:
    num = random.randint(1, 3) # 为了示例清晰，减少笑话数量
    print(f"--- Tool 'how_many_jokes' is called, returning: {num} ---")
    return num
```

## 拓展功能：列表输入与连续对话
- 支持输入列表，处理批量任务，更好管理输入
- 实现连续对话，保持上下文，实现多轮对话

### 列表输入
- 需求：需要给Agent包含上下文的对话历史；一次性提出多个问题/指令
- `Runner.run()`等支持传入消息列表，使用`messages`(包含`role`和`content`的字典列表)。

```python
messages = [
    {"role": "user", "content": "孔子的全名叫什么?"},
    {"role": "assistant", "content": "孔子的全名是孔丘，字仲尼。"}, # 可以包含历史
    {"role": "user", "content": "孟子的全名叫什么?"},
]
result = await Runner.run(agent, input=messages) # 直接传入列表
```

### 连续对话
- 需求：需要Agent能够记住并且保持上下文，实现多轮对话
- 实现：
    - `Runner.run()`执行结果会返回`RunResultStreaming`对象（`run_streamed`返回，可在其完成后访问）都有一个`to_input_list()`方法，用于将运行结果转换为消息列表格式。
    - `to_input_list()`会返回一个包含目前为止所有交互（用户输入、Agent回复、工具调用等）的消息列表，即为`messages`，这个列表在下一次`Runner.run()`的`input`传入

```python
# 第一轮
result1 = await Runner.run(agent, "金门大桥在哪个城市?")
print(result1.final_output)

# 第二轮
# 获取第一轮的完整历史，并加上用户的新问题
input_for_turn2 = result1.to_input_list() + [{"role": "user", "content": "它在哪个州?"}]
# 此处的`[{"role": "use...}]`是为了保持与传入的`messages`一致性
result2 = await Runner.run(agent, input_for_turn2)
print(result2.final_output)
```

### 终端人机交互（`run_demo_loop`）
- 快速在终端与Agent多轮交互测试，而不用手动管理`to_input_list()`
- 直接调用`Runner.run_demo_loop(agent)`，会进入一个循环，用户输入后Agent回复，持续交互，直到用户输入`exit`退出。
- `run_demo_loop`为异步函数，自动处理用户输入，调用`Runner.run_streamed`实现流式输出，自动维护对话历史。

```python
# ... (Agent定义) ...
async def main():
    await run_demo_loop(agent) # 启动交互式循环

if __name__ == "__main__":
    asyncio.run(main())
```

>建议补充示例`run_item_stream_event`与`run_demo_loop`之间的矛盾，从而揭示`run_demo_loop`的局限性
>因为其为封装隐藏事件流的处理细节函数，其内部原理是`async for event in result.stream_events()`循环，主要是为了打印`raw_response_event`（文本增量），以实现流畅的流式效果。它没有提供一个简单的方法来在其管理的循环中同时处理其他事件类型


## 区别：`hooks` & `run_item_stream_event`

| 维度                | run_streamed / run_item_stream                          | EventHooks (AgentHooks / RunHooks) 机制                |
|---------------------|---------------------------------------------------------|-------------------------------------------------------|
| 核心类型            | 异步事件流 (Async Event Stream)                         | 回调函数 (Callback Methods)                            |
| 获取方式            | 通过 `async for event in result.stream_events()` 循环获取 | 继承 AgentHooks 或 RunHooks 类并重写特定方法           |
| 主要目的            | 实时观察单次运行的所有细节，尤其是为了UI更新            | 响应生命周期中的关键事件，用于日志、监控、集成         |
| 粒度                | 非常细，包含LLM原始输出片段 (raw_response_event)等      | 较粗，只在预定义的关键节点触发（开始/结束/工具/交接） |
| 触发时机            | 事件发生时立刻产生                                      | 事件发生时框架调用对应方法                             |
| 作用范围            | 仅限于调用 run_streamed 的那一次特定运行                | AgentHooks: 附加到的特定Agent的所有运行； RunHooks: 某次特定运行中的所有Agent |
| 使用方式            | 在调用 run_streamed 的代码块中处理事件循环              | 定义钩子类，然后在创建Agent或调用Runner.run/run_sync时传入 |
| 数据访问            | 通过 event.type, event.data, event.item 访问事件信息     | 通过传递给钩子方法的参数访问信息 (如 agent, tool, output) |
| 修改能力            | 纯观察，无法干预Agent的执行流程                         | 理论上可以在钩子方法中修改状态（但不推荐用于控制流程），主要用于副作用 |